In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import tensorflow as tf
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential
import keras 
from keras.layers import Dense, Flatten, Conv1D,MaxPooling1D, Dropout,BatchNormalization,Embedding,Concatenate, Activation,Input
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K


<h1> Let's have some fun with Keras cooking

The goal was to mix Sequential and functional API and also 2 kind of model : Conv1D and Embedding.
You won't win the race, but the results are not ridiculous neither :-)

For those who are interested in creating a mix of API and Simple sequential model, it will be a simple example. 
So you will find :
- Sequential Embedding concatenated with Functional API Conv1D
- Sequential Conv1D concatenated with Functional API Embedding
- Sequential Conv1D concatenated with Sequential Embedding
- Functional API Conv1D concatenated with Functional API Embedding

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

<h2> Basic data preparation

In [ ]:
# Without labelencoding
target = pd.get_dummies(train['target'])
y = train['target']
X = train.iloc[:,1:-1]
test = test.iloc[:,1:]

# To avoid negative values (for embedding), 
# we just add 8 to all categories :
X = X + 8
test = test + 8
X.shape, test.shape, y.shape, target.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = y,
                                                    random_state = 41)

X_train.shape, y_test.shape, y_train.shape, y_test.shape

<h2> Compile & fit parameters

In [ ]:
es = callbacks.EarlyStopping(
                monitor = 'val_categorical_crossentropy', 
                min_delta = 0.00000001, 
                patience = 3,
                mode = 'min',
                baseline = None, 
                restore_best_weights = True,
                verbose = 1)

plateau  = callbacks.ReduceLROnPlateau(
                monitor = 'val_categorical_crossentropy',
                factor = 0.5, 
                patience = 2, 
                mode = 'min', 
                min_delt = 0.00000001,
                cooldown = 0, 
                min_lr = 1e-7,
                verbose = 1) 

metrics = [tf.keras.metrics.CategoricalCrossentropy()]
loss = tf.keras.losses.CategoricalCrossentropy(
                from_logits = False,
                label_smoothing = 0,
                reduction ="auto",
                name = "categorical_crossentropy")

<h2> Base models : Embedding & Conv1D with Sequential & functional API 

<h3> Functional API for Embedding

In [ ]:
inputs_API_Embedding = Input(shape = (50,), name = 'API_input_Embedding')
x = Embedding(80, 10, input_length = 50)(inputs_API_Embedding)
x = Flatten()(x)
x = Dense(80, activation ="relu")(x)
x = Dense(40, activation ="relu")(x)
x = Dense(20, activation ='relu')(x)
outputs_API_Embedding = Dense(4, activation ='relu')(x)

<h3> Sequential for Embedding

In [ ]:
model0 = Sequential()
model0.add(Embedding(80, 10, input_length = 50, input_shape = (50,)))
model0.add(Flatten())
model0.add(Dense(80, activation ='relu'))
model0.add(Dense(40, activation ='relu'))
model0.add(Dense(20, activation ='relu'))
model0.add(Dense(4, activation ='relu'))

<h3> Functional API for Conv1D

In [ ]:
inputs_API_Conv1D = Input(shape = (50,1), name = 'API_input_Conv1D') 
y = Conv1D(
            filters = 512, 
            kernel_size = 5,
            padding='same', 
            activation = 'relu',
            )(inputs_API_Conv1D)

y = MaxPooling1D(pool_size = 3)(inputs_API_Conv1D)
y = Flatten()(y)
y = Dense(64, activation = 'relu')(y)
outputs_API_Conv1D = Dense(4, activation ='relu')(y)

<h3> Sequential for Conv1D

In [ ]:
model3 = Sequential()
model3.add(Conv1D(
            filters = 256, 
            kernel_size = 5,
            padding = 'same', 
            activation = 'relu',
            input_shape = (50, 1)))

model3.add(MaxPooling1D(pool_size = 5))
model3.add(Flatten())
model3.add(Dense(64, activation ='relu'))
model3.add(Dense(4, activation ='softmax'))

<h3> Inputs and Outputs for Sequential models

In [ ]:
# We need to prepare inputs and outputs 
# for Sequential models to be able to concatenate with API later:

sequential_input_embedding = Input(shape = (50,), 
                                   name = 'Sequential_Embedding')
sequential_output_embedding = model0(sequential_input_embedding)

sequential_input_conv1D = Input(shape = (50,), 
                                name = 'Sequential_Conv1D')
sequential_output_conv1D = model0(sequential_input_conv1D)

<h2> Sequential Embedding + API Conv1D

In [ ]:
# Final step with concatenation of Embedding and Conv1D :
z = Concatenate(axis=1)([sequential_output_embedding, outputs_API_Conv1D])
out = Dense(4, activation = 'softmax', name = 'out')(z)

# Creation of the merged model :
model_merged = Model(
                    inputs = [sequential_input_embedding,inputs_API_Conv1D],
                    outputs = out, 
                    name = "model_merged")

# Compile and fit of the merged model :
model_merged.compile(
                    tf.keras.optimizers.Adam(learning_rate=0.0001),
                    loss = loss ,
                    metrics = metrics)
model_merged.fit(
                    {'Sequential_Embedding': X_train, 
                     'API_input_Conv1D': X_train},
                    {'out':y_train},
                    validation_data = ([X_test,X_test], y_test),
                    batch_size = 128,
                    epochs = 50,
                    verbose = 0,
                    callbacks = [es,plateau])

In [ ]:
print("Score for train.csv :",log_loss(target, model_merged.predict([X,X])))

<h2> Sequential Conv1D + API Embedding

In [ ]:
# Final step with concatenation of Embedding and Conv1D :
z = Concatenate(axis=1)([sequential_output_conv1D, outputs_API_Embedding])
out = Dense(4, activation = 'softmax', name = 'out')(z)

# Creation of the merged model :
model_merged = Model(
                    inputs=[sequential_input_conv1D,inputs_API_Embedding],
                    outputs=out,
                    name="model_merged")

# Compile and fit of the merged model :
model_merged.compile(
                    tf.keras.optimizers.Adam(learning_rate=0.0001),
                    loss = loss ,
                    metrics = metrics)
model_merged.fit(
                    {'Sequential_Conv1D':X_train, 
                     'API_input_Embedding':X_train},
                    {'out':y_train},
                    validation_data = ([X_test,X_test], y_test),
                    batch_size=128,
                    epochs=50,
                    verbose=0,
                    callbacks=[es,plateau])

In [ ]:
print("Score for train.csv :",log_loss(target, model_merged.predict([X,X])))

<h2> Sequential Conv1D + Sequential Embedding

In [ ]:
# Final step with concatenation of Embedding and Conv1D :
z = Concatenate(axis=1)([sequential_output_conv1D, sequential_output_embedding])
out = Dense(4, activation = 'softmax', name = 'out')(z)

# Creation of the merged model :
model_merged = Model(
                    inputs=[sequential_input_conv1D,sequential_input_embedding],
                    outputs=out, 
                    name="model_merged")

# Compile and fit of the merged model :
model_merged.compile(
                    tf.keras.optimizers.Adam(learning_rate=0.0001),
                    loss = loss ,
                    metrics = metrics)
model_merged.fit(
                    {'Sequential_Conv1D':X_train,
                     'Sequential_Embedding':X_train},
                    {'out':y_train},
                    validation_data = ([X_test,X_test], y_test),
                    batch_size=128,
                    epochs=50,
                    verbose=0,
                    callbacks=[es,plateau])

In [ ]:
print("Score for train.csv :",log_loss(target, model_merged.predict([X,X])))

<h2> API Conv1D + API Embedding

In [ ]:
# Final step with concatenation of Embedding and Conv1D :
z = Concatenate(axis=1)([outputs_API_Conv1D, outputs_API_Embedding])
out = Dense(4, activation = 'softmax', name = 'out')(z)

# Creation of the merged model :
model_merged = Model(
                    inputs=[inputs_API_Conv1D,inputs_API_Embedding],
                    outputs=out, 
                    name="model_merged")

# Compile and fit of the merged model :
model_merged.compile(
                    tf.keras.optimizers.Adam(learning_rate=0.0001),
                    loss = loss ,
                    metrics = metrics)
model_merged.fit(
                    {'API_input_Conv1D':X_train, 
                     'API_input_Embedding':X_train},
                    {'out':y_train},
                    validation_data = ([X_test,X_test], y_test),
                    batch_size=128,
                    epochs=50,
                    verbose=0,
                    callbacks=[es,plateau])

In [ ]:
print("Score for train.csv :",log_loss(target, model_merged.predict([X,X])))